In [4]:
import boto3
import json, uuid

In [ ]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [ ]:
!./setuprds.sh
!./setupdynamo.sh

In [8]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

In [9]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [15]:
function_list = local_lambda.list_functions()
# print(function_list)
print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

Function list:  ['create-account']


In [11]:
import uuid
test_account_dict = { 'clientId': 'something-or-other', 'ownerUserId': str(uuid.uuid4()), 'userFirstName': 'Luke', 'userFamilyName': 'Jordan' }
test_payload = json.dumps(test_account_dict)

In [18]:
local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', Payload=json.dumps(test_account_dict))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '132',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Fri, 24 May 2019 11:28:17 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7f4db149f7f0>}

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DeleteFunction operation: Function does not exist: create-account